In [1]:
!pip install bs4
!pip install requests
!pip install selenium
!pip install webdriver_manager

In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import warnings
import time

In [8]:
#'Enter Main Company link(Having 25 entries each) : 
complink = ['https://www.monsterindia.com/search/bridgetree-research-services-private-limited-55882-jobs-career?searchId=e9a6455e-6287-489d-8370-4eeaec265620']

#'Enter .CSV filename. ie, short Company name :       
filename = 'bridge.csv'     

#'Enter given email address :                                                                                                                                       
email_address = 'bridgetreeresearchservicesprivatelimitedjobsandcareer@sunisbrite.com'      

#'Enter Company UserName you entered :                                                                                   
username = 'bridge'                       

#'Enter Company name :                                                                                                                      
company_name = 'Bridgetree Research Services Private Limited' 

#'Check total number of jobs and enter here :
total_jobs = 19

In [9]:
string = complink[0]
loc = string.find('jobs-career')
loc = loc + 11
if total_jobs > 25:
    if total_jobs % 25 == 0:
        x = int(total_jobs//25)
    else:
        x = int(total_jobs//25)
        x = x+1

    for i in range(2, x+1):
        page = complink[0][0:loc] + f'-{i}' + complink[0][loc:]
        complink.append(page)
#print(complink)

In [10]:
def find_state(city):
    
    if city[0] == 'Mumbai' or city[0] == 'Pune' or city[0] == 'Navi Mumbai' or city[0] == 'Thar' or city[0] == 'Mumbai City' or city[0] == 'Aurangabad' or city[0] == 'Nagpur' or city[0] == 'Dhule':
        state = 'Maharashtra'
    elif city[0] == 'Noida' or city[0] == 'Lucknow':
        state = 'Uttar Pradesh'
    elif city[0] == 'Kochi' or city[0] == 'Cochin' or city[0] == 'Ernakulam' or city[0] == 'Kozhikode / Calicut' or city[0] == 'Thiruvananthapuram / Trivandrum' or city[0] == 'Palakkad':
        state = 'Kerala'
    elif city[0] == 'Bengaluru' or city[0] == 'Bangalore' or city[0] == 'Mysore' or city[0] == 'Bengaluru / Bangalore':
        state = 'Karnataka'
    elif city[0] == 'Chennai' or city[0] == 'Coimbatore':
        state  = 'Tamil Nadu'
    elif city[0] == 'Ahmedabad' or city[0] == 'Vadodara':
        state = 'Gujarat'
    elif city[0] == 'Gurugram' or city[0] == 'Gurgaon' or city[0] == 'Hisar':
        state = 'Haryana'
    elif city[0] == 'Kolkata' or city[0] == 'Salt':
        state = 'West Bengal'
    elif city[0] == 'Hyderabad' or city[0] == 'Secunderabad' or city[0] == 'Hyderabad / Secunderabad' or city[0] == 'Nalgonda':
        state = 'Telangana'
    elif city[0] == 'Udaipur' or city[0] == 'Jaipur':
        state = 'Rajasthan'
    elif city[0] == 'Panchkula' or city[0] == 'Chandigarh':
        state = 'Chandigarh'
    elif city[0] == 'Ludhiana' or city[0] == 'Amritsar':
        state = 'Punjab'
    elif city[0] == 'Motihari':
        state = 'Bihar'
    else:
        state = 'Delhi'

    return state

In [11]:
def identify_job_sector(x):
    
    if 'Human' in x or 'Recruitment' in x or 'Human Resources' in x or 'Admin' in x:
        x = 'HR Jobs'
    elif 'Digital Marketing' in x:
        x = 'Marketing Jobs'  
    elif 'Health' in x or 'Hospitals' in x or 'Healthcare' in x or 'Diagnostics' in x or 'Personal Care' in x:
        x = 'Health Care Jobs'
    elif 'Application' in x:
        x = 'Application Programming Jobs'
    elif 'Telecom Software' in x:
        x = 'Telecom Software Jobs'
    elif 'System' in x:
        x = 'System Programming Jobs'
    elif 'Site' in x:
        x = 'Site Engineering Jobs'
    elif 'Network' in x:
        x = 'Network administrator Jobs'
    elif 'Interior' in x:
        x = 'Interior Design Jobs'
    elif 'IT' in x or 'Information' in x:
        x = 'IT Jobs'
    elif 'Export' in x:
        x = 'Export Import'
    elif 'Graphic' in x:
        x = 'Graphic Designer Jobs'
    elif 'Hardwer' in x:
        x = 'Hardwer & Networking Jobs'
    elif 'BPO' in x:
        x = 'BPO / Call Centre'
    elif 'Business' in x:
        x = 'Business Intelligence Jobs'
    elif 'Client' in x:
        x = 'Client Server Jobs'
    elif 'Content' in x:
        x = 'Content Writing'
    elif 'Corporate' in x:
        x = 'Corporate Planning Jobs'
    elif 'Education' in x:
        x = 'Education Training'   
    elif 'Automobile' in x or 'Bank' in x or 'Analytics' in x or 'Airline' in x or 'Consultant' in x or 'DBA' in x or 'Ecommerce' in x or 'EDP' in x or 'ERP' in x or 'Film' in x or 'Hotel' in x or 'Legal' in x or 'Logistics' in x:
        x = x + ' Jobs'
    elif 'Mainframe' in x or 'Maintenance' in x or 'Marketing' in x or 'Middleware' in x or 'Mobile' in x or 'Packaging' in x or 'Pharma' in x or 'Secretary' in x or 'Security' in x or 'Shipping' in x or 'Testing' in x or 'VLSI' in x:
        x = x + ' Jobs'  
    else:
        x = 'IT Jobs'
    
    return x

In [12]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
data = []

for link in complink:
    #driver = webdriver.Edge(EdgeChromiumDriverManager().install())
    driver = webdriver.Chrome(ChromeDriverManager().install())

    for i in range(3):
        driver.get(link)
        time.sleep(2)

    resp = driver.page_source
    soup = BeautifulSoup(resp, 'html.parser')
    link_list = soup.select('[class="job-tittle"] h3 a')
    for link in link_list:
        url = link.get('href')
        
        driver.get(url)
        resp_info = driver.page_source
        soup_info = BeautifulSoup(resp_info, 'html.parser')

        job_details = soup_info.select('.job-details-wrapper')

        for info in job_details:

            job_title = (info.select('h1')[0].getText()).strip()
            location = (info.select('.loc.jd-loc a')[0].getText()).strip()
            try:
                if location == 'India':
                    location = (info.select('.loc.jd-loc a')[1].getText()).strip()
            except IndexError:  
                pass      
                
            CITY = location.replace(',', '',)
            city = location.split(',')
            state = find_state(city)
            experience = (info.select('.exp.col-xxs-12.col-sm-3.text-ellipsis span small')[0].getText()).strip().replace(' years', '').split('-')
            try:
                if experience[0] == 'Not Specified':
                    years = 'Fresh'
                elif int(experience[0]) == 0 or int(experience[1]) == 0:
                    years = 'Fresh'
                elif int(experience[0]) == 0 and int(experience[1]) == 1:
                    years = 'Less Than 1 Year'
                elif int(experience[0]) >= 8:
                    years = '8 Years +'
                elif int(experience[0]) >= 2 and int(experience[1]) <= 7:
                    years = experience[0] + ' Years'
                else: years = 'Fresh'
            except IndexError:
                years = 'Fresh'
                continue
            description = info.select('.job-description-content .jd-text')[0].getText().strip().replace('<br/>', '\n').replace(':', ':\n').replace('\t', '').replace('*', '\n*')
            description += '\nJob Details\n'
            job_sector = None
            key_skills = ''
            for count, details in enumerate(info.find_all('div', class_ ='job-detail-list')):
                dt_heading = details.find('div', class_ = 'dt-heading').text.replace('\t', '')
                dt_content = details.find('div', class_ = 'dt-content').text.replace('\t', '')
                description += f'\n{dt_heading}\n{dt_content}'
                #algorithm
                try:
                    if count == 2:
                        industry = details.find('div', class_ = 'dt-content').text.replace('\t', '').split('/')
                        if 'Other' in industry:
                            job_sector = 'IT Jobs'
                        else:
                            job_sector = industry[0]
                    job_sector=(job_sector.split(',')[0]).strip()
                    if count == 4:
                        description += '\nKey Skills\n'
                        for index, skill in enumerate(details.find_all('span', class_ = 'round-card mb5 grey-link')):
                            description += f' ➼ {skill.text}\n'
                            if index > 4:
                                break
                            key_skills += skill.text + ', '
                        continue
                except AttributeError:
                    continue
            if key_skills == "":
                key_skills= ' '
            
            employment = (info.select('#jobDets [class="card-panel"] > div:nth-child(1) .dt-content')[0].getText()).strip()
            #industry = (info.select('#jobDets [class="card-panel"] > div:nth-child(2) .dt-content')[0].getText()).strip()
            #function = (info.select('#jobDets [class="card-panel"] > div:nth-child(3) .dt-content')[0].getText()).strip()
            #skills = (info.select('#jobDets [class="card-panel"] > div:nth-child(4) .dt-content')[0].getText()).strip()
            min_salary, max_salary = '', ''
            try:
                salary = (info.select('.package')[0].getText()).strip()
                salary = salary.replace(' ', '').split('-')
                if salary[0] == 'NotSpecified':
                    min_salary += salary[1]
                else:
                    min_salary += salary[0]
                max_salary += salary[1]
            except:
                salary = ''
            if min_salary == '':
                min_salary= ' '
            if max_salary == '':
                max_salary= ' '
            temp = {
                'Job Title': job_title,
                'Location': location,
                'State': state,
                'CITY': CITY,
                'City': city[0],
                'Experience': years,
                'Description': description,
                'Employment': employment,
                'Job Sector':job_sector,
                'Key Skills': key_skills,
                #'Industry':industry,
                #'Function':function,
                #'Skills': skills,
                'Min Salary': min_salary,
                'Max Salary': max_salary
            }
            
            data.append(temp)
            print(f'  {len(data)} ====> {temp}')
    driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/root/.wdm/drivers/chromedriver/linux64/89.0.4389.23/chromedriver] found in cache


WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55a38d00b2b9 <unknown>


In [10]:
#Creating CSV file to store scrapped data
df = pd.DataFrame(data)
df.to_csv(filename,index=False,header=True)
data_list = pd.read_csv(filename).values.tolist()
len(data_list)

75

In [11]:
#Reading CSV file
data_list = pd.read_csv(filename).values.tolist()

#Posting Jobs
for i in range(len(data_list)):    
    #another_options = webdriver.EdgeOptions()
    another_options = webdriver.ChromeOptions()
    another_options.add_argument('--start-maximized')
    another_options.add_argument('--incognito')
    another_options.add_argument('--disable-gpu')
    another_options.add_argument('--disable-geolocation')
    another_options.add_argument('--disable-extensions')
    
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    #another_driver = webdriver.Edge(EdgeChromiumDriverManager().install())
    another_driver = webdriver.Chrome(ChromeDriverManager().install())
    another_driver.get('https://oxymars.com/india/post-new-job')
    time.sleep(2)

    # job title
    another_driver.find_element_by_id('ad-posting-title').send_keys(data_list[i][0])

    # job description
    another_driver.switch_to.frame("job_detail_ifr") # we need to change to frame in order to see the html elements/attribute
    another_driver.find_element_by_id('tinymce').send_keys(data_list[i][6]) # enter full description

    another_driver.switch_to.default_content() # switch to main page 

    #email address
    another_driver.find_element_by_name('reg_user_email').send_keys(email_address)
    
    #username   
    another_driver.find_element_by_name('reg_user_uname').send_keys(username)
    
    #company name
    another_driver.find_element_by_name('pt_user_organization').send_keys(company_name)

    #job sector
    sector = identify_job_sector(data_list[i][8])
    another_driver.find_element_by_id('job-sector-selectized').send_keys(sector)
    another_driver.find_element_by_id('job-sector-selectized').send_keys(Keys.ENTER)

    #job type
    another_driver.find_element_by_id('job-type-selectized').send_keys(data_list[i][7])
    another_driver.find_element_by_id('job-type-selectized').send_keys(Keys.ENTER)

    # skills here
    another_driver.find_element_by_xpath('//*[@id="job-skills"]/li/input').send_keys(data_list[i][9])

    # salary
    another_driver.find_element_by_name('job_salary').send_keys(data_list[i][10])
    another_driver.find_element_by_name('job_max_salary').send_keys(data_list[i][11])

    #other info
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[1]/div/div/div[1]/input').send_keys('Others')
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[1]/div/div/div[1]/input').send_keys(Keys.ENTER)
    
    # experiece
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[2]/div/div/div[1]/input').send_keys(data_list[i][5])
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[2]/div/div/div[1]/input').send_keys(Keys.ENTER)

    #gender
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[3]/div/div/div[1]/input').send_keys('Any')
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[3]/div/div/div[1]/input').send_keys(Keys.ENTER)
    
    #qualifications 
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[4]/div/div/div[1]/input').send_keys("Degree Bachelor")
    another_driver.find_element_by_xpath('//*[@id="job-posting-form"]/div[2]/ul/li[4]/div/div/div[1]/input').send_keys(Keys.ENTER)

    #country   
    another_driver.find_element_by_xpath('//*[@id="countryId-selectized"]').send_keys('India')
    another_driver.find_element_by_xpath('//*[@id="countryId-selectized"]').send_keys(Keys.ENTER)
    time.sleep(5)

    #state    
    another_driver.find_element_by_xpath('//*[@id="stateId-selectized"]').send_keys(data_list[i][2]) 
    another_driver.find_element_by_xpath('//*[@id="stateId-selectized"]').send_keys(Keys.ENTER)
    time.sleep(5)

    #city
    if data_list[i][2] == 'Delhi':
        pass
    elif data_list[i][2] == 'Orrisa':
        pass
    else:    
        another_driver.find_element_by_xpath('//*[@id="cityId-selectized"]').send_keys(data_list[i][4])
        another_driver.find_element_by_xpath('//*[@id="cityId-selectized"]').send_keys(Keys.ENTER)
    
    CITY=data_list[i][3]
    complete_address = f'{CITY}, India'
    if 'India' in CITY:
        complete_address = CITY
    another_driver.find_element_by_name('jobsearch_field_location_address').send_keys(complete_address)

    another_driver.find_element_by_name('terms_cond_check').click()
    another_driver.find_element_by_css_selector('input[type="submit"]').click()

    print(f'Posting {i+1} done in oxymars.com :)')
    time.sleep(2)
    another_driver.quit()



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\nikit\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Posting 1 done in oxymars.com :)




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\nikit\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Posting 2 done in oxymars.com :)




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\nikit\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


...

Posting 8 done in oxymars.com :)




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\nikit\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


Posting 9 done in oxymars.com :)




====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [C:\Users\nikit\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


KeyboardInterrupt: 